In [1]:
import tensorflow as tf
import numpy as np
import math

%load_ext autoreload
%autoreload 2

In [2]:
import util

address = '../../data/'
file_names = {}
file_names['images'] = 'small_data.npy'
file_names['subs'] = 'small_subredditlabels'
file_names['dict'] = 'small_subredditIndex'
file_names['nsfw'] = 'small_nsfwlabels'
data, dictionary = util.import_dataset(address, file_names)

In [3]:
print(data.X_train.shape)
print(data.X_val.shape)
print(data.X_test.shape)
print(data.y_train.shape)
print(data.y_val.shape)
print(data.y_test.shape)

(800, 128, 128, 3)
(100, 128, 128, 3)
(100, 128, 128, 3)
(800,)
(100,)
(100,)


## Run the Model

In [10]:
from model import Model, lazy_property

class GoogleNet(Model):
    def __init__(self, model_config):
        Model.__init__(self, model_config)
    
    def inception(self, input_layer, num_1x1, num_3x3_reduce, num_3x3, num_double_3x3_reduce, num_double_3x3, 
                  pool_type, proj_size, strided):
        strides = [2, 2] if strided else [1, 1]  # last layer strides (before concatenation)

        if num_1x1 > 0:
            inception_1_conv1 = tf.layers.conv2d(input_layer, num_1x1, [1, 1], strides=strides, padding="SAME")
            inception_1_bn1 = tf.layers.batch_normalization(inception_1_conv1, training=self.is_training_placeholder)
            inception_1 = tf.nn.relu(inception_1_bn1)

        inception_2_conv1 = tf.layers.conv2d(input_layer, num_3x3_reduce, [1, 1], strides=[1, 1], padding="SAME", activation=tf.nn.relu)
        inception_2_conv2 = tf.layers.conv2d(inception_2_conv1, num_3x3, [3, 3], strides=strides, padding="SAME")
        inception_2_bn1 = tf.layers.batch_normalization(inception_2_conv2, training=self.is_training_placeholder)
        inception_2 = tf.nn.relu(inception_2_bn1)

        inception_3_conv1 = tf.layers.conv2d(input_layer, 64, [1, 1], strides=[1, 1], padding="SAME", activation=tf.nn.relu)
        inception_3_conv2 = tf.layers.conv2d(inception_3_conv1, 96, [3, 3], strides=[1, 1], padding="SAME", activation=tf.nn.relu)
        inception_3_conv3 = tf.layers.conv2d(inception_3_conv2, 96, [3, 3], strides=strides, padding="SAME")
        inception_3_bn1 = tf.layers.batch_normalization(inception_3_conv3, training=self.is_training_placeholder)
        inception_3 = tf.nn.relu(inception_3_bn1)

        inception_4_pool1 = tf.nn.pool(input_layer, [3, 3], pool_type, "SAME", strides=strides)
        if proj_size == 0:
            inception_4 = tf.nn.relu(inception_4_pool1)  # pass through layer if proj_size is 0
        else:
            inception_4_conv1 = tf.layers.conv2d(inception_4_pool1, proj_size, [1, 1], padding="SAME")
            inception_4_bn1 = tf.layers.batch_normalization(inception_4_conv1, training=self.is_training_placeholder)
            inception_4 = tf.nn.relu(inception_4_bn1)

        if num_1x1 > 0:
            inception_out = tf.concat([inception_1, inception_2, inception_3, inception_4], -1)
        else:
            inception_out = tf.concat([inception_2, inception_3, inception_4], -1)
        return inception_out
    
    @lazy_property
    def prediction(self):
        conv_1 = tf.layers.conv2d(self.X_placeholder, 64, [7, 7], strides=[2, 2], padding="SAME", activation=tf.nn.relu)
        pool_1 = tf.layers.max_pooling2d(conv_1, [3, 3], [2, 2], "SAME")
        norm_1 = tf.layers.batch_normalization(pool_1, training=self.is_training_placeholder)
        conv_2 = tf.layers.conv2d(norm_1, 192, [3, 3], strides=[1, 1], padding="SAME", activation=tf.nn.relu)
        pool_2 = tf.layers.max_pooling2d(conv_2, [3, 3], [2, 2], "SAME")
        norm_2 = tf.layers.batch_normalization(pool_2, training=self.is_training_placeholder)

        inception_1a = self.inception(norm_2, 64, 64, 64, 64, 96, "AVG", 32, False)
        inception_1b = self.inception(inception_1a, 64, 64, 96, 64, 96, "AVG", 64, False)
        inception_1c = self.inception(inception_1b, 0, 128, 160, 64, 96, "MAX", 0, True)
        inception_2a = self.inception(inception_1c, 224, 64, 96, 96, 128, "AVG", 128, False)
        inception_2b = self.inception(inception_2a, 192, 96, 128, 96, 128, "AVG", 128, False)
        inception_2c = self.inception(inception_2b, 160, 128, 160, 128, 160, "AVG", 128, False)
        inception_2d = self.inception(inception_2c, 96, 128, 192, 160, 192, "AVG", 128, False)
        inception_2e = self.inception(inception_2d, 0, 128, 192, 192, 256, "MAX", 0, True)
        inception_3a = self.inception(inception_2e, 352, 192, 320, 160, 224, "AVG", 128, False)
        inception_3b = self.inception(inception_3a, 352, 192, 320, 192, 224, "MAX", 128, False)

        # The following pooling size is changed from the original paper due to different starting image sizes
        pool_3 = tf.nn.pool(inception_3b, [4, 4], "AVG", "VALID", strides=[1, 1])
        if self.config.keep_prob < 1.0:
            pool_3 = tf.nn.dropout(pool_3, self.config.keep_prob)
        y_out = tf.layers.dense(pool_3, 20)
        return y_out[:, 0, 0, :]

In [11]:
from config import ModelConfig, TrainConfig

tf.reset_default_graph()

model_config = ModelConfig()
train_config = TrainConfig(print_every=1, num_epochs=2)
model = GoogleNet(model_config)
sess = tf.Session()
model.train(data, sess, train_config)

---------------------------------------------------------
Batch 1/13 of epoch 1 finished in 4.177517 seconds
Batch 2/13 of epoch 1 finished in 0.305029 seconds
Batch 3/13 of epoch 1 finished in 0.300214 seconds
Batch 4/13 of epoch 1 finished in 0.292665 seconds
Batch 5/13 of epoch 1 finished in 0.292057 seconds
Batch 6/13 of epoch 1 finished in 0.293832 seconds
Batch 7/13 of epoch 1 finished in 0.293571 seconds
Batch 8/13 of epoch 1 finished in 0.293946 seconds
Batch 9/13 of epoch 1 finished in 0.291217 seconds
Batch 10/13 of epoch 1 finished in 0.290580 seconds
Batch 11/13 of epoch 1 finished in 0.293666 seconds
Batch 12/13 of epoch 1 finished in 0.292295 seconds
Batch 13/13 of epoch 1 finished in 0.189416 seconds
Epoch 1 training finished in 7.606733 seconds
train accuracy:6.2%
val accuracy:4.0%
---------------------------------------------------------
Batch 1/13 of epoch 2 finished in 0.295148 seconds
Batch 2/13 of epoch 2 finished in 0.289791 seconds
Batch 3/13 of epoch 2 finished 

KeyboardInterrupt: 